# Vectorization of Samples
To conserve resources and prepare the samples for final ingest, the last thing we need to do is turn all of the samples into vectors. This will be done using a custom tokenizer similar to that in tokenization.ipynb in MalDroid_feature_engineering repo. It will take the sample.apk.json files as a string and convert them to index ints. Due to the large size of our training dataset and the potential for the future computation of additional n-grams, performance and reusability will be optimized. The vocabulary.txt file from the aforementioned repo has been copied to local.
## Algorithm Process (for sample in samples; given: regex delimiter from vocab)
1. Fetch sample and load as dict, extract list of behaviors
2. Call multisort function, ordering by id then ts 
3. For behavior in sorted list, drop unused keys
4. Cast each dict in sorted list to string and add to str list of behaviors
5. Initalize vector with SOA index, For behavior in list, call tokenizer function passing vector: 
6. Append vector with SOB index, init prev_matchEnd=0
7. For match in finditer(delim, behavior):
8. If match.start() - prev_matchEnd > 2, append vector with UNK index
9. Append vector with match.lastgroup set prev_matchEnd to match.end(), next match
10. Append vector with EOB index, return vector, next behavior
11. Append vector with EOA index, save to .tfrecord under *class*/*hash*.tfrecord, next sample

In [ ]:
import json
import tensorflow as tf
import numpy as np

In [ ]:
def sort_behaviors(raw_behaviors):
    raw_behaviors.sort(key = lambda x: x['low'][0]['id'])
    raw_behaviors.sort(key = lambda x: float(x['low'][0]['ts']))
    #check to ensure this correctly parses str to float
    return raw_behaviors

In [ ]:
def strip_unused_keys(behavior_list, unused_keys):
    behavior_index = 0
    for behavior in behavior_list:
        behavior = {key: value for key, value in behavior.items() if key not in unused_keys}
        sub_behavior_index = 0
        for sub_behavior in behavior['low']:
            sub_behavior = {sub_key: sub_value for sub_key, sub_value in sub_behavior.items() if sub_key not in unused_keys}
            behavior['low'][sub_behavior_index] = sub_behavior
            sub_behavior_index += 1
        behavior_list[behavior_index] = behavior
        behavior_index += 1
    return behavior_list

In [ ]:
def tokenize(vector, behavior):
    vector = np.array(vector)
    

In [ ]:
mal_classes = ['adware', 'banking', 'riskware', 'sms']
error_hashes = []
parent_dir = 'X:\\MITRE\\MalDroid Data\\MalDroid_feature_engineering\\'

for mal_class in mal_classes:
    for sample_folder in os.listdir(parent_dir + mal_class + '\\'):
        with open(parent_dir + mal_class + '\\' + sample_folder + '\\sample_for_analysis.apk.json') as sample_path:
            try:
                sample_behaviors = json.loads(sample_path)['behaviors']['dynamic']['host']
            except:
                error_hashes.append(sample_folder)
                sample_behaviors *= 0 
                continue

        sorted_behaviors = sort_behaviors(sample_behaviors)
        sample_behaviors *= 0 
        # lists are cleared after useage to preserve memory resources

        stripped_behaviors = strip_unused_keys(sorted_behaviors, [])
        sorted_behaviors *= 0 

        string_behaviors = [for ast.literal_eval(behavior) in stripped_behaviors] 
        # check to ensure this preserves json syntax (ex "" not '' in keys)
        stripped_behaviors *= 0 

        vectorized_sample = [1]
        for behavior in string_behaviors:
            np.concatenate([vectorized_sample, tokenize(vectorized_sample, behavior)])